# Prepare Query Files

This notebook contains code to gather the queries that I want to use for my assessment experiment.

## Imports

In [1]:
import pandas as pd
import numpy as np

## Paths

In [2]:
msmarco_dir = "../data/msmarco_files/"
anserini_dir = "../data/anserini_output/"
output_dir = "../data/output/"

## Load Data

CHANGE THE FOLLOWING FILENAMES

In [3]:
query_filename = 'queries.dev.small.tsv'
bm25_filename = 'run_development_top100.tsv'
bert_filename = 'bert_run_development_top100.tsv'
label_filename = "label_analysis_dev_small.tsv"
relevance_filename = 'qrels.dev.small.tsv'

In [4]:
query_subset = pd.read_csv(msmarco_dir + query_filename,delimiter='\t',encoding='utf-8', header=None)
query_subset.columns = ['query_id', 'query_text']

In [5]:
query_subset.shape

(6980, 2)

In [6]:
bm25_rankings = pd.read_csv(anserini_dir + bm25_filename,delimiter='\t',encoding='utf-8', header=None)
bm25_rankings.columns = ['query_id', 'passage_id', 'ranking']

In [6]:
bm25_rankings.shape

(808500, 3)

In [7]:
bm25_rankings[bm25_rankings['passage_id'] == 7564409]

,query_id,passage_id,ranking
74826,1074807,7564409,27
596174,750111,7564409,23


In [8]:
labels_df = pd.read_csv(output_dir + label_filename,delimiter='\t',encoding='utf-8',header=None)
labels_df.columns = ['query_id', 'passage_id', 'bm25_label', 'bert_label', 'bm25_num_label', 'bert_num_label', 'label_comparison', 'query_text', 'passage_text']

In [9]:
labels_df.shape

(4460, 9)

In [10]:
passage_df = pd.read_csv(msmarco_dir + 'collection.tsv',delimiter='\t',encoding='utf-8',header=None)
passage_df.columns = ['passage_id','passage_text']

In [11]:
bert_rankings = pd.read_csv(output_dir + bert_filename, delimiter='\t',encoding='utf-8',header=None)
bert_rankings.columns = ['query_id', 'passage_id', 'bm25_rank', 'query_text', 'passage_text', 'bert_score', 'bert_rank']

In [12]:
bert_rankings.shape

(473568, 7)

In [13]:
rel_labels_df = pd.read_csv(msmarco_dir + relevance_filename, delimiter='\t',encoding='utf-8',header=None)
rel_labels_df.columns = ['query_id', 'unknown_label', 'passage_id', 'relevance_label']

In [14]:
idsWithRelLabel = labels_df['query_id'].tolist()
idsWithoutRelLabel = list(np.unique(bm25_rankings[~bm25_rankings['query_id'].isin(idsWithRelLabel)]['query_id'].tolist()))

In [15]:
len(idsWithoutRelLabel + idsWithRelLabel)

6980

## Combine Data

In [16]:
bm25_output_df = pd.DataFrame(idsWithoutRelLabel + idsWithRelLabel)
bm25_output_df.columns = ['query_id']
bm25_output_df.shape

(6980, 1)

In [17]:
bm25_output_df = bm25_output_df.merge(bm25_rankings[bm25_rankings['ranking'] < 21],how='left',on=['query_id'])
bm25_output_df.shape

(139596, 3)

In [18]:
bm25_output_df = bm25_output_df.merge(query_subset,how='left',on=['query_id'])
bm25_output_df.shape

(139596, 4)

In [19]:
bm25_output_df = bm25_output_df.merge(passage_df,how='left',on=['passage_id'])
bm25_output_df.shape

(139596, 5)

In [20]:
bm25_output_df = bm25_output_df.merge(labels_df[['query_id','bm25_label']],how='left',on=['query_id'])
print(np.unique(bm25_output_df['bm25_label'].tolist()))
bm25_output_df.shape

['high' 'low' 'medium' 'nan']


(139596, 6)

In [21]:
bm25_output_df['bm25_label'] = bm25_output_df['bm25_label'].fillna('outside scope')
print(np.unique(bm25_output_df['bm25_label'].tolist()))
bm25_output_df.shape

['high' 'low' 'medium' 'outside scope']


(139596, 6)

In [22]:
bm25_output_df = bm25_output_df.merge(rel_labels_df[['query_id','passage_id','relevance_label']],how='left',on=['query_id','passage_id'])
bm25_output_df.shape

(139596, 7)

In [23]:
bm25_output_df['relevance_label'] = bm25_output_df['relevance_label'].fillna(0.0)

In [24]:
vc = bm25_output_df['bm25_label'].value_counts()
for k, v in vc.items():
    print(k)
    print(v/20)

high
3248.8
outside scope
2520.0
medium
1063.0
low
148.0


#### BERT output dataframe

In [25]:
bert_output_df = pd.DataFrame(idsWithRelLabel)
bert_output_df.columns = ['query_id']
bert_output_df.shape

(4460, 1)

In [26]:
bert_output_df = bert_output_df.merge(bert_rankings[bert_rankings['bert_rank'] < 21][['query_id','passage_id','bert_rank']],how='left',on=['query_id'])
bert_output_df.shape

(89841, 3)

In [27]:
bert_output_df = bert_output_df.sort_values(by=['query_id','bert_rank'])
bert_output_df.shape

(89841, 3)

Be aware: bert sometimes gave multiple query passage combinations the same rank. This means that instead of only top 20 you sometimes get top 21, 22 or more. For my experiment I only need 20 passages thus I ranked them on bert rank and later on I will only take the first 20 passages. 

In [28]:
ids = []
vc = bert_output_df['query_id'].value_counts()
for k, v in vc.items():
    if v > 20:
        ids.append(k)

In [30]:
bert_output_df = bert_output_df.merge(query_subset,how='left',on=['query_id'])
bert_output_df.shape

(89841, 4)

In [31]:
bert_output_df = bert_output_df.merge(passage_df,how='left',on=['passage_id'])
bert_output_df.shape

(89841, 5)

In [32]:
bert_output_df = bert_output_df.merge(labels_df[['query_id','bert_label']],how='left',on=['query_id'])
print(np.unique(bert_output_df['bert_label'].tolist()))
bert_output_df.shape

['high' 'low' 'medium']


(89841, 6)

I only took the queries with the msmarco relevant label inside top 100 and reranked those with bert. That is why in this dataframe you wont find any passages with "outside scope" label.

In [33]:
bert_output_df = bert_output_df.merge(rel_labels_df[['query_id','passage_id','relevance_label']],how='left',on=['query_id','passage_id'])
bert_output_df.shape

(89841, 7)

In [34]:
bert_output_df['relevance_label'] = bert_output_df['relevance_label'].fillna(0.0)
bert_output_df.shape

(89841, 7)

In [35]:
import math
vc = bert_output_df['bert_label'].value_counts()
for k, v in vc.items():
    print(k)
    print(math.floor(v/20))

high
4175
medium
313
low
3


In [36]:
bert_output_df[bert_output_df['bert_label'] == 'high'].head(10)

,query_id,passage_id,bert_rank,query_text,passage_text,bert_label,relevance_label
0,2,4339068,1,Androgen receptor define,"The androgen receptor (AR), also known as NR3C...",high,1.0
1,2,378374,2,Androgen receptor define,An androgen (from Greek andro meaning male hum...,high,0.0
2,2,3575680,3,Androgen receptor define,Androgen (from Greek andro meaning male human ...,high,0.0
3,2,6285817,4,Androgen receptor define,The term SARMS stands for âSelective Androge...,high,0.0
4,2,1001873,5,Androgen receptor define,The AR gene provides instructions for making a...,high,0.0
5,2,7496503,6,Androgen receptor define,The androgen receptor gene is more than 90 kb ...,high,0.0
6,2,6920058,7,Androgen receptor define,The AR gene provides instructions for making a...,high,0.0
7,2,3634076,8,Androgen receptor define,SARMS or Selective Androgen Receptor Modulator...,high,0.0
8,2,6285819,9,Androgen receptor define,SARMS stands for selective androgen receptor m...,high,0.0
9,2,4542570,10,Androgen receptor define,"Antiandrogens, also known as androgen antagoni...",high,0.0


## Write to TSV file

#### BM25

In [37]:
bm25_output_dir = output_dir + "experiment_files/tsv/bm25/"

In [38]:
labels = ['outside scope', 'low', 'medium', 'high']

In [39]:
for label in labels:
    temp_df = bm25_output_df[bm25_output_df['bm25_label'] == label]
    query_ids = np.unique(temp_df['query_id'].tolist())
    for query_id in query_ids:
        query_top20_df = temp_df[temp_df['query_id'] == query_id]
        if query_top20_df.shape[0] == 20:
            if label == 'outside scope':
                filename = bm25_output_dir + 'outside_scope/' + str(query_id) + '.tsv'
            else:
                filename = bm25_output_dir + label + '/' + str(query_id) + '.tsv'
            query_top20_df.to_csv(filename,sep="\t", header=False,index=False)

#### BERT

In [40]:
bert_output_dir = output_dir + "experiment_files/tsv/bert/"

In [41]:
labels = ['low', 'medium', 'high']

In [42]:
for label in labels:
    temp_df = bert_output_df[bert_output_df['bert_label'] == label]
    query_ids = np.unique(temp_df['query_id'].tolist())
    for query_id in query_ids:
        query_top20_df = temp_df[temp_df['query_id'] == query_id]
        if query_top20_df.shape[0] == 20:
            filename = bert_output_dir + label + "/" + str(query_id) + '.tsv'
            query_top20_df.to_csv(filename,sep="\t",header=False,index=False)
        elif query_top20_df.shape[0] > 20:
            query_top20_df = query_top20_df.iloc[:20]
            filename = bert_output_dir + label + "/" + str(query_id) + '.tsv'
            query_top20_df.to_csv(filename,sep="\t",header=False,index=False)

## Write to JSON

#### BM25

In [43]:
bm25_output_dir = output_dir + "experiment_files/json/bm25/"

In [44]:
labels = ['outside scope', 'low', 'medium', 'high']

In [45]:
for label in labels:
    temp_df = bm25_output_df[bm25_output_df['bm25_label'] == label]
    query_ids = np.unique(temp_df['query_id'].tolist())
    for query_id in query_ids:
        query_top20_df = temp_df[temp_df['query_id'] == query_id]
        if query_top20_df.shape[0] == 20:
            if label == 'outside scope':
                filename = bm25_output_dir + 'outside_scope/' + str(query_id) + '.json'
            else:
                filename = bm25_output_dir + label + '/' + str(query_id) + '.json'
            query_top20_df.to_json(filename,orient='records')

In [46]:
example_file = pd.read_json(bm25_output_dir + "outside_scope/1102335.json")

In [47]:
example_file.head(1)

,bm25_label,passage_id,passage_text,query_id,query_text,ranking,relevance_label
0,outside scope,7826611,Your question why don't people view a car sole...,1102335,why do people buy cars,1,0


#### BERT

In [48]:
bert_output_dir = output_dir + "experiment_files/json/bert/"

In [49]:
labels = ['low', 'medium', 'high']

In [50]:
for label in labels:
    temp_df = bert_output_df[bert_output_df['bert_label'] == label]
    query_ids = np.unique(temp_df['query_id'].tolist())
    for query_id in query_ids:
        query_top20_df = temp_df[temp_df['query_id'] == query_id]
        if query_top20_df.shape[0] == 20:
            filename = bert_output_dir + label + '/' + str(query_id) + '.json'
            query_top20_df.to_json(filename,orient='records')
        elif query_top20_df.shape[0] > 20:
            query_top20_df = query_top20_df.iloc[:20]
            filename = bert_output_dir + label + '/' + str(query_id) + '.json'
            query_top20_df.to_json(filename,orient='records')